# LLMs and their tools

## Objective:
LLMs don't execute tools directly - they only decide which tools to use and parse arguments. The actual tool execution happens in separate code.

This notebook compares two approaches for integrating tools with LLMs:

1. Manual Tool Chaining (explicit parsing/execution)

2. llm.bind_tools() (automatic tool-call binding).

With bind_tools(), this process is automated through structured outputs, while manual chaining gives you control by parsing text responses like 'TOOL: calculator|2+2' and routing to functions yourself.

We’ll explore the trade-offs between these methods, including how each interacts with RunnableWithMessageHistory for stateful conversations. Below is a high-level comparison:

| Aspect               | Manual Tool Chaining                        | `bind_tools()`                                   |
|----------------------|---------------------------------------------|--------------------------------------------------|
| **Code Clarity**     | More complex, error-prone parsing           | Cleaner, automatic JSON-based tool calls         |
| **Tool Execution**   | Manual (logic in prompts/code)              | Decoupled: LLM suggests calls, code executes     |
| **Memory Support**   | Native via message history wrapper          | Requires manual tool-result logging              |
| **Flexibility**      | Full control over workflow                  | Structured (limited by API design)               |
| **Best For**         | Custom workflows, complex logic             | Rapid development, standardized tooling          |

In [ ]:
! pip install langchain==0.3.25 langgraph==0.4.5 langchain-openai==0.3.18 python-dotenv==1.1.0 gradio==5.17.1

  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 6.3 MB/s eta 0:00:00
Using cached python_multipart-0.0.20-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 8.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0
    Uninstalling websockets-15.0:
      Successfully uninstalled websockets-15.0
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.1
    Uninstalling python-dotenv-1.0.1:
      Successfully uninstalled python-dotenv-1.0.1
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninsta

## Setup

In [ ]:
## Common Setup
import os
from datetime import datetime
# Load environment variables
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda

import logging

load_dotenv()
logger = logging.getLogger()
logger.setLevel(logging.INFO)


# Shared tool definitions
@tool
def calculator_tool(expression: str) -> float:
    """Calculate mathematical expressions safely."""
    try:
        # Simple safe evaluation (in production, use a proper math parser)
        result = eval(expression.replace("^", "**"))
        return float(result)
    except:
        return "Error: Invalid expression"


@tool
def get_current_time(format_str: str = "%Y-%m-%d %H:%M:%S") -> str:
    """Get current date and time in specified format."""
    return datetime.now().strftime(format_str)


## Lets compare Manual Chaining vs Bind tools

## Approach 1 : Manual Chaining

In [ ]:
# Initialize LLM
llm_for_manual_chain = ChatOpenAI(temperature=0)

# Step 1: Define prompt template for tool selection
prompt_for_llm_for_manual_chain = ChatPromptTemplate.from_template(
    """Analyze the user's question and decide if tools are needed.
    Available tools:
    - calculator_tool: For math operations (input: math expression)
    - get_current_time: Returns current time (input: time format string)

    Respond EXACTLY in one of these formats:

    TOOL: calculator_tool|2+2
    TOOL: get_current_time|%Y-%m-%d
    ANSWER: direct_answer (e.g., "ANSWER: The capital of France is Paris")

    User question: {question}"""
)

# Tool mapping dictionary
tool_map = {
    'calculator_tool': calculator_tool,
    'get_current_time': get_current_time
}

def execute_tools_for_manual_chain(response: AIMessage) -> str:
    """Execute tools based on LLM response or return direct answer."""
    content = response.content
    print(content)

    if content.startswith("TOOL:"):
        try:
            # Parse tool call
            _, tool_call = content.split(":", 1)
            tool_name, args = tool_call.strip().split("|")

            # Execute tool
            tool = tool_map[tool_name]
            result = tool.invoke(args.strip())
            return f"{result}"

        except Exception as e:
            return f"Error executing tool: {str(e)}"

    elif content.startswith("ANSWER:"):
        # Return direct answer
        return content.split(":", 1)[1].strip()

    else:
        return "I couldn't process your request properly."

# Step 2: Create the chain
manual_chain = (
    prompt_for_llm_for_manual_chain
    | llm_for_manual_chain
    | RunnableLambda(execute_tools_for_manual_chain)
)

In [ ]:
# Test 1: Date query
print("Query: What's today's date?")
response = manual_chain.invoke({"question": "What's today's date?"})
print(f"Response: {response}\n")

# Test 2: Math query
print("Query: What's 4+4?")
response = manual_chain.invoke({"question": "What's 4+4?"})
print(f"Response: {response}\n")

# Non tool call
print("Query: What's captial of UK")
response3 = manual_chain.invoke({"question":"What's captial of UK"})
print(response3)

Query: What's today's date?
TOOL: get_current_time|%Y-%m-%d
Response: 2025-06-10

Query: What's 4+4?
TOOL: calculator_tool|4+4
Response: 8.0

Query: What's captial of UK
ANSWER: The capital of the UK is London.
The capital of the UK is London.


## Approach 2 : LLM for bind tools

In [ ]:
import json

llm_with_bind_tools = ChatOpenAI(temperature=0)
tools = [calculator_tool, get_current_time]
prompt_for_llm_with_bind_tools = ChatPromptTemplate.from_template(
    """Analyze the user's question and use if tools are needed. Otherwise just answer the question
    User question: {question}"""
)

# Tool mapping dictionary
tool_map = {
    'calculator_tool': calculator_tool,
    'get_current_time': get_current_time
}

def execute_tools_for_bind_tools(response: AIMessage) -> str:
    """Execute tools based on LLM response or return direct answer."""
    tool_calls = response.additional_kwargs.get("tool_calls", [])

    for call in tool_calls:
        function_call = call['function']
        # Correct way to get tool name and arguments
        tool_name = function_call["name"]
        tool_args = json.loads(function_call["arguments"])

        # Run the tool manually with keyword args unpacked
        if tool_name == "calculator_tool":
            output = calculator_tool.invoke(input=tool_args)
        elif tool_name == "get_current_time":
            output = get_current_time.invoke(input=tool_args)
        else:
            output = "Tool not found."
        return output
    return response.content


# Step 2: Create the chain
bind_tools_chain = (
    prompt_for_llm_with_bind_tools
    | llm_with_bind_tools.bind_tools(tools) | RunnableLambda(execute_tools_for_bind_tools)
)

In [ ]:
# Test 1: Date query
print("Query: What's today's date?")
response = bind_tools_chain.invoke({"question": "What's today's date?"})
print(f"Response: {response}\n")

# Test 2: Math query
print("Query: What's 4+4?")
response = bind_tools_chain.invoke({"question": "What's 4+4?"})
print(f"Response: {response}\n")

# Non tool call
print("Query: What's captial of UK")
response3 = bind_tools_chain.invoke({"question":"What's captial of UK"})
print(response3)

Query: What's today's date?
Response: 2025-06-10 23:51:58

Query: What's 4+4?
Response: 8.0

Query: What's captial of UK
The capital of the UK is London.


## 3. Adding Memory to Manual Chain and Bind Tools Chain

### Adding Memory to Manual Chain

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

class ManualToolChaining:
    """Explicit tool handling with manual chaining and session memory."""

    def __init__(self):
        """Initialize with the pre-built manual chain."""
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a helpful assistant that MUST follow the exact response format.

        Available tools:
        - calculator_tool: For mathematical calculations (any math expression)
        - get_current_time: For current date/time (use format like %Y-%m-%d %H:%M:%S)

        CRITICAL: You MUST respond in EXACTLY one of these formats:

        For tool usage:
        Always use all caps TOOL
        TOOL: calculator_tool|2+2
        TOOL: get_current_time|%Y-%m-%d %H:%M:%S

        For direct answers:
        ANSWER: Your direct response here

        EXAMPLES:
        - User asks "What's the date?" → Respond: "TOOL: get_current_time|%Y-%m-%d"
        - User asks "What's 5+3?" → Respond: "TOOL: calculator_tool|5+3"
        - User asks "What's the capital of France?" → Respond: "ANSWER: The capital of France is Paris"
        - User asks about previous conversation → Look at the conversation history and respond: "ANSWER: [summary of what they asked]"

        DO NOT give conversational responses like "I can help you with that" - use the exact format above."""),
                    MessagesPlaceholder(variable_name="history"),
                    ("human", "{question}")
                ])

        self.manual_chain = self.prompt | llm_for_manual_chain | RunnableLambda(execute_tools_for_manual_chain)
        self.store = {}

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        """Get or create session history for the given session ID."""
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    def process_message(self, message: str, session_id: str = "default") -> str:
        """Process message with manual tool chaining and memory."""
        try:
            chain_with_memory = RunnableWithMessageHistory(
                self.manual_chain,
                self.get_session_history,
                input_messages_key="question",
                history_messages_key="history"
            )

            # Invoke the chain
            response = chain_with_memory.invoke(
                {"question": message},
                config={"configurable": {"session_id": session_id}}
            )

            return response

        except Exception as e:
            return f"Error processing message: {str(e)}"

In [ ]:
manual_chain_with_memory = ManualToolChaining()
# Test 1: Date query
print("Query: What's today's date?")
response = manual_chain_with_memory.process_message(message ="What's date", session_id ="manual_session")
print(f"Response: {response}\n")

# Test 2: Math query
print("Query: What's 4+4?")
response = manual_chain_with_memory.process_message(message ="What's 4+4?", session_id ="manual_session")
print(f"Response: {response}\n")

# Non tool call
print("Query: Add 4 to previous answer")
response = manual_chain_with_memory.process_message(message ="Add 4 to previous answer", session_id ="manual_session")
print(response)

Query: What's today's date?
TOOL: get_current_time|%Y-%m-%d %H:%M:%S
Response: 2025-06-10 23:52:02

Query: What's 4+4?
TOOL: calculator_tool|4+4
Response: 8.0

Query: Add 4 to previous answer
TOOL: calculator_tool|8+4
12.0


### Adding Memory to Bind Tools Chain

In [ ]:
import json
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage

class AutomaticToolBinding:
    """Auto tool handling + manual tool execution + resumed response."""

    def __init__(self):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful assistant with tools. Use them when needed."),
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}")
        ])
        self.bind_tools_chain = self.prompt |  llm_with_bind_tools.bind_tools(tools)
        self.store = {}

    def get_session_history(self, session_id: str) -> BaseChatMessageHistory:
        if session_id not in self.store:
            self.store[session_id] = InMemoryHistory()
        return self.store[session_id]


    def process_message(self, message: str, session_id: str = "default") -> str:

        chain_with_memory = RunnableWithMessageHistory(
            self.bind_tools_chain ,
            self.get_session_history,
            input_messages_key="question",
            history_messages_key="history"
        )

        # Step 1: Get model's initial response
        response = chain_with_memory.invoke(
            {"question": message},
            config={"configurable": {"session_id": session_id}}
        )

        tool_calls = response.additional_kwargs.get("tool_calls", [])
        history = self.get_session_history(session_id=session_id)

        if tool_calls:
            tool_messages = []

            for call in tool_calls:
                function_call = call['function']
                # Correct way to get tool name and arguments
                tool_name = function_call["name"]
                tool_args = json.loads(function_call["arguments"])
                tool_id = call["id"]

                # Run the tool manually with keyword args unpacked
                if tool_name == "calculator_tool":
                    output = calculator_tool.invoke(input=tool_args)
                elif tool_name == "get_current_time":
                    output = get_current_time.invoke(input=tool_args)
                else:
                    output = "Tool not found."

                # Wrap tool output into ToolMessage
                tool_msg = ToolMessage(tool_call_id=tool_id, content=str(output))
                tool_messages.append(tool_msg)
                history.add_message(tool_msg)

            # Step 2: Feed tool outputs back into model
            followup = llm_with_bind_tools.invoke([
                HumanMessage(content=message),
                response,
                *tool_messages
            ])
            return followup.content
        else:
            return response.content

In [ ]:
bind_tools_chain_with_memory = AutomaticToolBinding()
# Test 1: Date query
print("Query: What's today's date?")
response = bind_tools_chain_with_memory.process_message(message ="What's date", session_id ="manual_session")
print(f"Response: {response}\n")

# Test 2: Math query
print("Query: What's 4+4?")
response = bind_tools_chain_with_memory.process_message(message ="What's 4+4?", session_id ="manual_session")
print(f"Response: {response}\n")

# Non tool call
print("Query: Add 4 to previous answer")
response = bind_tools_chain_with_memory.process_message(message ="Add 4 to previous answer", session_id ="manual_session")
print(response)

Query: What's today's date?
Response: The current date is June 10, 2025.

Query: What's 4+4?
Response: 4 + 4 equals 8.

Query: Add 4 to previous answer
The result of adding 4 to the previous answer is 12.
